<a href="https://colab.research.google.com/github/jjgp/colab/blob/master/notebooks/ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install git+https://github.com/jjgp/colab

  Cloning https://github.com/jjgp/colab to /tmp/pip-req-build-slee_p7b
  Running command git clone -q https://github.com/jjgp/colab /tmp/pip-req-build-slee_p7b
  Created wheel for colab: filename=colab-0.0.1-py3-none-any.whl size=1405 sha256=ae5cfb29aaeb413a944165ce797005346f33d959af4dbe6722f6d56072b765d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-08bgyf1c/wheels/ae/51/e1/6f52a739c14d371b54949fb83481b088970c32219da1e72293
Successfully built colab
